In [ ]:
import sys
import random
import math
import matplotlib.pyplot as plt
PATH_TO_AIMA = "../aima-python"
if PATH_TO_AIMA not in sys.path:
    sys.path.insert(0,PATH_TO_AIMA)
from search import *
import timeit
# from notebook import psource, heatmap, gaussian_kernel, show_map, final_path_colors, display_visual, plot_NQueens
import walls

In [ ]:
# Global maze declaration
maze_array = None
time_measure = {
    "BFS": [],
    "DLS": [],
    "ASL": [],
    "ASM": [],
    "GL": [],
    "GM": [],
    "HC": [],
    "pacman_initials": [],
    "goal_positions": [],
}

## Definição do Objetivo do Trabalho

O  Pacman  encontra-se  em  um  labirinto  composto  pelas paredes, 3 fantasmas estáticos em posições pré-definidas, um ponto objetivo e dots. O objetivo desse trabalho é levar o Pacman até o ponto objetivo.
O Pacman  se  move  de  acordo  com  vizinhança-4,  a  cada  ação executada  ele  coleta  um  dot  do  labirinto  e  revisitar  posições nao é permitido. 

O labirinto é representado como uma grid de tamanho variável. As paredes do labirinto são intransponíveis e o ambiente é completamente observável e determinıstico.

## Problem Modeling

Para modelar o problema utilizamos a classe "Problem" presente na biblioteca AIMA (https://github.com/aimacode/aima-python) em Python 3.

Definimos as seguintes propriedades do problema:
- O labirinto é global
- Os estados representam a posição (x,y) do Pacman no labirinto.
- As possíveis ações são tuplas (x,y) nas direções Oeste, Norte, Sul e Leste, que são testadas nessa ordem.
- A posição inicial do Pacman é gerada aleatoriamente dentro das possíveis posições¹.
- A posição do objetivo é gerada aleatoriamente dentro das possíveis posições¹.
- As posições dos fantasmas são geradas aleatoriamente dentro das possíveis posições¹.

¹ posições possíveis são posições representadas por dots na geração do labirinto.

A modelagem foi feita definindo as funções abaixo.
- actions: Determina a partir da posição (x,y) do Pacman e do estado do labirinto quais ações são possíveis de serem executadas.
- result: Retorna o novo estado do Pacman após executar uma ação.
- goaltest: Testa se o Pacman chegou no objetivo.
- pathcost: Determina o custo para cada ação executada. Para o nosso caso, cada passo dado pelo Pacman tem custo 1.
- value: Retorna quantos dots foram coletados. É utilizada no algoritmo de busca local para ser maximizada.



In [ ]:
class PacmanMazeProblem(Problem):
    """The Pacman Maze Problem."""
    
    def __init__(self, pac_position, goal_position):
        Problem.__init__(self, pac_position, goal_position)
        
    def actions(self, state):
        """The possible actions are the neighbors of a given node
        except if the neighbor is a ghost or a wall"""        
        actions = []
        y, x, _ = state
        directions = [         (0, -1),
                      (-1, 0),           (1,  0),
                                (0, 1)           ]
        for position in directions:
            possible_pos = [y+position[1], x+position[0]]
            if possible_pos[1] < 0:
                possible_pos[1] = maze_array.width-1
            elif possible_pos[1] == maze_array.width:
                possible_pos[1] = 0
            possible_pos = tuple(possible_pos)
            maze_value = maze_array.maze[possible_pos]
            if maze_value == CellType.DOT:
                actions.append(possible_pos)
        return actions

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state."""
        global maze_array
        
        maze_array.maze[state[0]][state[1]] = CellType.EMPTY
        _, _, dots = state
        state = (action[0], action[1], dots+1)
        maze_array.maze[state[0]][state[1]] = CellType.EMPTY
        
        return state
    
    def goal_test(self, state):
        """Return True if the state is a goal."""
        y, x, _ = state
        
        return y == self.goal[0] and x == self.goal[1]

    def path_cost(self, c, state1, action, state2):
        """
        Every action is a step costing one
        """
        return c + 1
    
    def value(self, state):
        """For optimization problems, each state has a value. Hill Climbing
        and related algorithms try to maximize this value."""
        _, _, dots = state
        return dots

### Classe Maze

A classe Maze é responsável por criar o labirinto.

Dado um tamanho de largura e altura, uma grid é criada com uma configuração aleatória de paredes e dots. Dentre as posições dos dots 5 são escolhidas aleatoriamente para representarem as posições dos 3 fantasmas, a posição inicial do Pacman e a posição do objetivo.

O código de geração do labirinto "wall.py" foi retirado do github (https://github.com/shaunlebron/pacman-mazegen)

In [ ]:
class CellType:
    EMPTY = 0
    WALL = 1
    DOT = 3
    GHOST = 2

class Maze:
    def __init__(self, width, height):
        self.width = width
        self.height = height
    
    def __getitem__(self, position):
        return self.maze[position]
    
    def __setitem__(self, new_value, position):
        self.maze[position] = new_value
    
    def show_maze(self, solution):
        for point in solution:
            plt.scatter(point[1], point[0], color='red')

        plt.imshow(self.maze, interpolation='nearest', cmap=plt.cm.get_cmap("cubehelix", 4))
        plt.scatter(self.pac_position[1], self.pac_position[0], color='yellow')
        plt.scatter(self.goal_position[1], self.goal_position[0], color='orange')
        plt.axis('off')
        plt.show()
              
    def maze_to_array(self, maze_str, width, height):
        """
        Function that converts string maze to numpy array
        """
        maze = []
        for line in str(maze_str).splitlines():
            line = line+line[::-1]
            if line:
                line = list(line)
                for item in range(len(line)):
                    if line[item] == "|":
                        line[item] = CellType.WALL
                    else:
                        line[item] = CellType.DOT
# If we want to do wihtout 2 for:
#                 line = np.array(list(line))
#                 line = np.where(line == '|', CellType.WALL, CellType.DOT)
                maze.append(line)
        return np.array(maze) 
    
    def maze_generator(self):
        """
        Function that generates a random maze
        Input: height and width of the maze
        Output: array maze
        """
        half_width = self.width//2
        maze = np.zeros((self.height, half_width))
        maze[0,:] = 1
        maze[:,0] = 1
        maze[-1,:] = 1
        maze[self.height//2-2:self.height//2+2, half_width-3::] = 1
        
        gap_number = random.randint(0, 3)
        for i in range(gap_number):
            maze[(i+1)*self.height//(gap_number+1), 0] = 0
        
        maze_str = []
        for line in maze:
            for item in line:
                if item == 1:
                    maze_str.append('|')
                else:
                    maze_str.append('.')
            maze_str.append('\n')
        maze_str = ''.join(maze_str)

        maze = walls.Map(half_width, self.height, maze_str)
        # generate map by adding walls until there's no more room
        while maze.add_wall_obstacle(extend=True):
            pass
        
        maze_array = self.maze_to_array(maze, half_width, self.height)
        
        # insert ghosts to the in random positions of the maze
        for i in range(3):
            valid_positions = np.argwhere(maze_array==CellType.DOT)
            position = random.choice(valid_positions)
            maze_array[position[0],position[1]] = CellType.GHOST
            print("Ghost #%d position:"%i, tuple(position))
        
        # pacman initial position is a tuple of 3 values: (x, y, dots)
        # where dots are the total dots eaten by pacman
        valid_positions = np.argwhere(maze_array==CellType.DOT)
        position = random.choice(valid_positions)
        position = np.append(position, [0])
        self.pac_position = tuple(position)
        self.goal_position = tuple(random.choice(valid_positions))
        print("Pacman initial positon: ", self.pac_position)
        print("Goal position: ", self.goal_position)
        
        # Saves the generated maze configuration
        self.maze = maze_array
        y,x, _ = self.pac_position
        self.maze[(y,x)] = CellType.EMPTY
        unique, counts = np.unique(self.maze, return_counts="True")
        print("Total number of dots in the maze: ", dict(zip(unique, counts))[CellType.DOT])
        self.original_maze = self.maze.copy()


### Heurísticas

As heurísticas são utilizadas nos algoritmos de busca informada.

Foram definidas duas heurísticas:
- distância linear
- distância de manhatan

In [ ]:
def linear(node):  
    Y, X, _ = node.state
    (gY, gX) = maze_array.goal_position
        
    return int(math.sqrt((gY-Y)**2+(gX-X)**2))

def manhatan(node):  
    Y, X, _ = node.state
    (gY, gX) = maze_array.goal_position
        
    return (abs(gY-Y)+abs(gX-X))

In [ ]:
# Set the size of the Maze
MAZE_WIDTH = 50
MAZE_HEIGHT = 50

global maze_array
maze_array = Maze(MAZE_WIDTH, MAZE_HEIGHT)
maze_array.maze_generator()

In [ ]:
time_measure["pacman_initials"].append((maze_array.pac_position[0], maze_array.pac_position[1]))
time_measure["goal_positions"].append(maze_array.goal_position)

## Uninformed Search Methods

In [ ]:
problem = PacmanMazeProblem(maze_array.pac_position, maze_array.goal_position)

In [ ]:
# global maze_array
maze_array.maze = maze_array.original_maze.copy()
solution = breadth_first_graph_search(problem)
print("Path Solution Found: ", solution.solution())
print("Cost of the path found: ", solution.path_cost)

print("\nFinal maze:")
maze_array.show_maze(solution.solution())

In [ ]:
global maze_array
maze_array.maze = maze_array.original_maze.copy()
solution = depth_limited_search(problem)
try:
    print("Path Solution Found: ", solution.solution())
    print("Cost of the path found: ", solution.path_cost)
    print("\nFinal maze:")
    maze_array.show_maze(solution.solution())
except:
    print("Path not found with depth limited = 50")

In [ ]:
%%timeit -o
############### Breadth First Search #####################

global maze_array
maze_array.maze = maze_array.original_maze.copy()
solution = breadth_first_graph_search(problem)

In [ ]:
time_measure["BFS"].append(_.average)

In [ ]:
%%timeit -o
############### Depth Limited Search #####################
global maze_array
maze_array.maze = maze_array.original_maze.copy()
solution = depth_limited_search(problem)

In [ ]:
time_measure["DLS"].append(_.average)

## Informed Search Methods

In [ ]:
global maze_array
maze_array.maze = maze_array.original_maze.copy()
solution = astar_search(problem, linear)

print("Path Solution Found: ", solution.solution())
print("Cost of the path found: ", solution.path_cost)
print("\nFinal maze:")
maze_array.show_maze(solution.solution())

In [ ]:
global maze_array
maze_array.maze = maze_array.original_maze.copy()
solution = astar_search(problem, manhatan)

print("Path Solution Found: ", solution.solution())
print("Cost of the path found: ", solution.path_cost)
print("\nFinal maze:")
maze_array.show_maze(solution.solution())

In [ ]:
global maze_array
maze_array.maze = maze_array.original_maze.copy()
solution = best_first_graph_search(problem, linear)

print("Path Solution Found: ", solution.solution())
print("Cost of the path found: ", solution.path_cost)
print("\nFinal maze:")
maze_array.show_maze(solution.solution())

In [ ]:
global maze_array
maze_array.maze = maze_array.original_maze.copy()
solution = best_first_graph_search(problem, manhatan)

print("Path Solution Found: ", solution.solution())
print("Cost of the path found: ", solution.path_cost)
print("\nFinal maze:")
maze_array.show_maze(solution.solution())

In [ ]:
%%timeit -o
############### A* with h() = linear distance #####################

global maze_array
maze_array.maze = maze_array.original_maze.copy()
solution = astar_search(problem, linear)

In [ ]:
time_measure["ASL"].append(_.average)

In [ ]:
%%timeit -o
############### A* with h() = manhatan distance #####################
global maze_array
maze_array.maze = maze_array.original_maze.copy()
solution = astar_search(problem, manhatan)

In [ ]:
time_measure["ASM"].append(_.average)

In [ ]:
%%timeit -o
########### Greedy Best First with f() = linear distance #########

global maze_array
maze_array.maze = maze_array.original_maze.copy()
solution = best_first_graph_search(problem, linear)

In [ ]:
time_measure["GL"].append(_.average)

In [ ]:
%%timeit -o
########### Greedy Best First with f() = manhatan distance #########

global maze_array
maze_array.maze = maze_array.original_maze.copy()
solution = best_first_graph_search(problem, manhatan)

In [ ]:
time_measure["GM"].append(_.average)

## Local Search Method

In [ ]:
maze_array.maze = maze_array.original_maze.copy()
solution = hill_climbing(problem)
print("Number of eaten dots: ", solution[2])
maze_array.show_maze([(solution[0], solution[1])])

In [ ]:
%%timeit -o
############### Hill Climbing Local Search #####################
global maze_array
maze_array.maze = maze_array.original_maze.copy()
solution = hill_climbing(problem)

In [ ]:
time_measure["HC"].append(_.average)

In [ ]:
print(time_measure)

In [ ]:
# def compare_searchers(problems, header,
#                       searchers=[breadth_first_graph_search,
#                                  depth_limited_search,
#                                  [best_first_graph_search, 'manhatan'],
#                                  [best_first_graph_search, 'linear'],
#                                  [astar_search, 'manhatan'],
#                                  [astar_search, 'linear'],
#                                  hill_climbing]):
    
#     def do(searcher, problem):
#         p = InstrumentedProblem(problem)
#         if searcher == [astar_search, 'manhatan']:
#             searcher[0](p, manhatan)
#         elif searcher == [astar_search, 'linear']:
#             searcher[0](p, linear)
#         elif searcher == [best_first_graph_search, 'manhatan']:
#             searcher[0](p, manhatan)
#         elif searcher == [best_first_graph_search, 'linear']:
#             searcher[0](p, linear)
#         else:
#             searcher(p)
#         return p

#     table = [[name(s)] + [do(s, p) for p in problems] for s in searchers]
#     print_table(table, header)

In [ ]:
# headers = ["search algorithms", "1"]
# compare_searchers([problem], headers)

## Referências